In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
import io, math, os, sys
from IPython.core.display import HTML

import numpy as np

# Install daltonlens if necessary
try:
    from daltonlens import convert, simulate
except ImportError:
    !pip install -q daltonlens
    from daltonlens import convert, simulate

# SVG takes 4x5 matrices for rgba + offset
def printColblindlyMatrix(name, m, severity):
    m = severity*m + (1.0-severity)*np.identity(3)
    print(name)
    print(f"values=\"{m[0,0]:.5f},{m[0,1]:.5f},{m[0,2]:.5f},0,0 {m[1,0]:.5f},{m[1,1]:.5f},{m[1,2]:.5f},0,0 {m[2,0]:.5f},{m[2,1]:.5f},{m[2,2]:.5f},0,0 0,0,0,1,0\"")
    print()

# Introduction

The goal is to generate the precomputed matrices needed by ColorBlindly, in the linearRGB space. They are meant to be used with `color-interpolation-filters="linearRGB"`.

The chosen model is Viénot, Brettel and Mollon 1999 _Digital video colourmaps for checking the legibility of displays by dichromats. Color Research & Application_ because it reduces to a single matrix in linear RGB space.

Anomalous trichromacy is handled via linear interpolation with the original image.

An alternative with a single matrix would be the Machado 2009 model: https://www.inf.ufrgs.br/~oliveira/pubs_files/CVD_Simulation/CVD_Simulation.html 

The Viénot 1999 single matrix model is not great for tritanopia, ideally we'd use the Brettel, Viénot and Mollon 1997 model _Computerized simulation of color appearance for dichromats_. But it requires blending two projection matrices, not sure how we can do this with SVG filters.

For more information about CVD simulation models, see https://daltonlens.org/opensource-cvd-simulation/ and https://daltonlens.org/understanding-cvd-simulation/ .

In [24]:
simulator = simulate.Simulator_Vienot1999(convert.LMSModel_sRGB_SmithPokorny75())

simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.PROTAN, severity=1.0)
printColblindlyMatrix("protanopia", simulator.cvd_linear_rgb, 1.0)
printColblindlyMatrix("protanomaly", simulator.cvd_linear_rgb, 0.6)

simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.DEUTAN, severity=1.0)
printColblindlyMatrix("deuteranopia", simulator.cvd_linear_rgb, 1.0)
printColblindlyMatrix("deuteranomaly", simulator.cvd_linear_rgb, 0.6)

simulator.simulate_cvd(np.zeros((1,1,3), dtype=np.uint8), simulate.Deficiency.TRITAN, severity=1.0)
printColblindlyMatrix("tritanopia", simulator.cvd_linear_rgb, 1.0)
printColblindlyMatrix("tritanomaly", simulator.cvd_linear_rgb, 0.6)

protanopia
values="0.10889,0.89111,-0.00000,0,0 0.10889,0.89111,0.00000,0,0 0.00447,-0.00447,1.00000,0,0 0,0,0,1,0"

protanomaly
values="0.46533,0.53467,-0.00000,0,0 0.06533,0.93467,0.00000,0,0 0.00268,-0.00268,1.00000,0,0 0,0,0,1,0"

deuteranopia
values="0.29031,0.70969,-0.00000,0,0 0.29031,0.70969,-0.00000,0,0 -0.02197,0.02197,1.00000,0,0 0,0,0,1,0"

deuteranomaly
values="0.57418,0.42582,-0.00000,0,0 0.17418,0.82582,-0.00000,0,0 -0.01318,0.01318,1.00000,0,0 0,0,0,1,0"

tritanopia
values="1.00000,0.15236,-0.15236,0,0 0.00000,0.86717,0.13283,0,0 -0.00000,0.86717,0.13283,0,0 0,0,0,1,0"

tritanomaly
values="1.00000,0.09142,-0.09142,0,0 0.00000,0.92030,0.07970,0,0 -0.00000,0.52030,0.47970,0,0 0,0,0,1,0"

